In [10]:
import scrapy
import time, threading
from scrapy_selenium import SeleniumRequest
from scrapy.selector import Selector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, TimeoutException
from shutil import which
from openpyxl import load_workbook

wb = load_workbook('BD_CADASTRO_NUMERADO_AGO_TESTE.xlsx')
ws, ws2 = wb['Fundos'], wb['Fundos_Cota']
lista_cnpj = []

for row in ws.iter_rows(values_only=True):
    cnpj = row[1]
    lista_cnpj.append(cnpj)

for row in ws2.iter_rows(values_only=True):
    cnpj = row[1] 
    lista_cnpj.append(cnpj)

print("Coleta de CNPJs finalizada")

lista_cnpj_override = "01.608.573/0001-65"

dados_diarios_list = []

def init_driver():
    options = webdriver.ChromeOptions()

    options.add_argument("--headless=new")

    driver = webdriver.Chrome(executable_path=which('chromedriver'), options=options)

    url='https://cvmweb.cvm.gov.br/swb/default.asp?sg_sistema=fundosreg'

    driver.get(url)

    return driver

def get_info(cnpj, index):
    try:
        print(f"Thread {index} iniciada para o CNPJ {cnpj}")
        driver = init_driver()
        print(f"Driver da Thread {index} iniciado com sucesso")

        #pesquisa e troca para o frame correto sem precisar declarar uma variável nova no processo
        WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
        )

        search_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[@id='txtCNPJNome']"))
        )

        search_input.clear()
        #search_input.send_keys(lista_cnpj[0])
        search_input.send_keys(cnpj)
        search_input.send_keys(Keys.ENTER)
        print(f"Thread {index} pesquisou o CNPJ {cnpj}")

        find_name_to_store = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[@id='ddlFundos__ctl1_Linkbutton4']"))
        )
        stored_name = find_name_to_store.text
        find_name_to_store.click()

        driver.switch_to.default_content()

        #pesquisa e troca para o frame correto sem precisar declarar uma variável nova no processo
        WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
        )

        dados_diarios = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[@id='Hyperlink2']"))
        )
        dados_diarios.click()
        print(f"Thread {index} clicou em dados diários")

        driver.switch_to.default_content()

        #pop-up com mensagem irrelevante aparece e precisa ser fechado.
        try:
            alert = WebDriverWait(driver, 3).until(EC.alert_is_present())
            alert.accept()
        except:
            pass

        WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
        )

        dropdown_data_pesquisa = Select(WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
        ))

        print(f"Thread {index} iniciando coleta de dados diários")
        #dados_diarios_list = []
        for i, _ in enumerate(dropdown_data_pesquisa.options):
            driver.switch_to.default_content()

            WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
            )

            dropdown_data_pesquisa = Select(WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
            ))
            dropdown_data_pesquisa.select_by_index(i)

            name_selected_dropdown = Select(WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
            ))
            name_selected_dropdown = name_selected_dropdown.first_selected_option.get_attribute("value")
            
            linhas_tabela = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, "//table[@id='dgDocDiario']//tr[position()>1]"))
            )
            linhas_validas = [linha for linha in linhas_tabela if linha.find_element(By.XPATH, "td[2]").text.strip()] 
            
            if linhas_validas:
                ultima_linha = linhas_validas[-1]
                dados = ultima_linha.find_elements(By.XPATH, "td")
                dados_diarios = {
                    "Cnpj": cnpj,
                    "Nome": stored_name,
                    "Mês": name_selected_dropdown,
                    "Dia": dados[0].text.strip(),
                    "Quota": dados[1].text.strip(),
                    #"Captação no Dia": dados[2].text.strip(),
                    #"Resgate no Dia": dados[3].text.strip(),
                    "Patrimônio Líquido": dados[4].text.strip(),
                    #"Total da Carteira": dados[5].text.strip(),
                    "Número de Cotistas": dados[6].text.strip(),
                    #"Data da Próxima Informação do PL": dados[7].text.strip(),
                }
                dados_diarios_list.append(dados_diarios)
        #dropdown.select_by_visible_text('02/2025')
        print(f"Thread {index} finalizou coleta de dados diários")
        driver.quit()
    except TimeoutException:
        print(f"Erro na Thread {index}")
        driver.quit()
        print(f"Thread {index} reiniciando...")
        get_info(cnpj, index)

list_cnpj = ["58.878.941/0001-02", "36.248.874/0001-00", "51.017.442/0001-81", "57.270.020/0001-08"]

threads = []
for index, cnpj in enumerate(list_cnpj):
    thread = threading.Thread(target=get_info, args=(cnpj, index+1,))
    threads.append(thread)
    thread.start()

# Aguardando todas as threads terminarem
for thread in threads:
    thread.join()

print("Todas as Threads foram finalizadas")




# thread1 = threading.Thread(target=get_info, args=("58.878.941/0001-02",))
# thread2 = threading.Thread(target=get_info, args=("36.248.874/0001-00",))
# thread3 = threading.Thread(target=get_info, args=("51.017.442/0001-81",))
# thread4 = threading.Thread(target=get_info, args=("57.270.020/0001-08",))

# def repeat_thread(thread):
#     while True:
#         try:
#             thread.start()
#             break
#         except:
#             repeat_thread(thread)

# repeat_thread(thread1)
# repeat_thread(thread2)
# repeat_thread(thread3)
# repeat_thread(thread4)

# thread1.join()
# thread2.join()
# thread3.join()
# thread4.join()

Coleta de CNPJs finalizada
Thread 1 iniciada para o CNPJ 58.878.941/0001-02
Thread 2 iniciada para o CNPJ 36.248.874/0001-00
Thread 3 iniciada para o CNPJ 51.017.442/0001-81
Thread 4 iniciada para o CNPJ 57.270.020/0001-08
Driver da Thread 2 iniciado com sucesso
Driver da Thread 4 iniciado com sucessoDriver da Thread 3 iniciado com sucesso

Thread 2 pesquisou o CNPJ 36.248.874/0001-00
Thread 4 pesquisou o CNPJ 57.270.020/0001-08
Thread 3 pesquisou o CNPJ 51.017.442/0001-81
Driver da Thread 1 iniciado com sucesso
Thread 1 pesquisou o CNPJ 58.878.941/0001-02
Thread 4 clicou em dados diários
Thread 3 clicou em dados diários
Thread 1 clicou em dados diários
Thread 4 iniciando coleta de dados diários
Thread 3 iniciando coleta de dados diários
Thread 1 iniciando coleta de dados diários
Thread 1 finalizou coleta de dados diários
Erro na Thread 2
Thread 4 finalizou coleta de dados diários
Thread 3 finalizou coleta de dados diários
Thread 2 reiniciando...
Thread 2 iniciada para o CNPJ 36.248.87

In [11]:
import pandas as pd
testing = pd.DataFrame(dados_diarios_list)
testing_sorted = testing.sort_values(by='Cnpj', ignore_index=True)
testing_sorted

,Cnpj,Nome,Mês,Dia,Quota,Patrimônio Líquido,Número de Cotistas
0,36.248.874/0001-00,051 CRÉDITO 90 FUNDO DE INVESTIMENTO FINANCEIR...,02/2025,14,"1,5760233","375.417.776,16",55
1,36.248.874/0001-00,051 CRÉDITO 90 FUNDO DE INVESTIMENTO FINANCEIR...,01/2025,31,"1,566572","373.149.451,52",55
2,36.248.874/0001-00,051 CRÉDITO 90 FUNDO DE INVESTIMENTO FINANCEIR...,12/2024,31,"1,5486839","372.740.265,23",54
3,36.248.874/0001-00,051 CRÉDITO 90 FUNDO DE INVESTIMENTO FINANCEIR...,11/2024,29,"1,5316007","368.876.093,12",56
4,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,02/2025,14,"1.286,77293445","15.089.017,33",6
5,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,01/2025,31,"1.277,94733581","14.985.526,18",6
6,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,12/2024,31,"1.253,2158189","13.935.343,64",6
7,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,11/2024,29,"1.204,61210793","13.126.145,44",6
8,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,10/2024,31,"1.195,32948236","13.024.996,62",6
9,57.270.020/0001-08,1295 ESTRUTURADO FUNDO DE INVESTIMENTO MULTIME...,02/2025,14,"1,02854268","112.621.354,07",1
